## 同时使用两个摄像头

In [1]:
import traitlets
from traitlets.config.configurable import SingletonConfigurable
import numpy as np
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import sys
sys.path.append('..')
from opencv_course.video import Camera

image_left = None
image_right = None

def bgr8_to_jpeg_left(value, quality=10):
    global image_left, image_right
    image_left = cv2.imencode('.jpg', value)[1]
    if image_right is not None:
        images = [image_left, image_right]
        stitcher = cv2.Stitcher_create()
        (status, stitched) = stitcher.stitch(images)
        return bytes(stitched)
    return bytes(image_left)

def bgr8_to_jpeg_right(value, quality=10):
    global image_left, image_right
    image_right = cv2.imencode('.jpg', value)[1]
    if image_left is not None:
        images = [image_left, image_right]
        stitcher = cv2.Stitcher_create()
        (status, stitched) = stitcher.stitch(images)
        return bytes(stitched)
    return bytes(image_right)

## 打开板载摄像头

In [2]:
camera = Camera(width=320, height=240, fps=10, is_usb=False)

image = widgets.Image(format='jpeg', width=244, height=244)  # this width and height doesn't necessarily have to match the camera

traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg_left)

display(image)

w:320.0, h:240.0, fps:10.0, brightness:0.0, contrast:0.0, zoom:0.0
h, w: 240, 320


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## 打开USB摄像头

In [3]:
usb_camera = Camera(width=320, height=240, fps=10, is_usb=True)

image2 = widgets.Image(format='jpeg', width=244, height=244)  # this width and height doesn't necessarily have to match the camera

traitlets.dlink((usb_camera, 'value'), (image2, 'value'), transform=bgr8_to_jpeg_right)

display(image2)

w:2304.0, h:1536.0, fps:2.0, brightness:131.0, contrast:141.0, zoom:0.0
h, w: 240, 320


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "../opencv_course/video.py", line 63, in _capture_frames
    self.value = image
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 585, in __set__
    self.set(obj, value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 574, in set
    obj._notify_trait(self.name, old_value, new_value)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1139, in _notify_trait
    type='change',
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 1176, in notify_change
    c(change)
  File "/usr/local/lib/python3.6/dist-packages/traitlets/traitlets.py", line 347, in _update
    self._transform(change.new))
  File "<ipython-input-1-510122f4bf

error: OpenCV(4.1.0) /home/bbot/opencv41/opencv-4.1.0/modules/imgproc/src/resize.cpp:3721: error: (-215:Assertion failed) inv_scale_x > 0 in function 'resize'


In [ ]:
import traitlets
import ipywidgets
import subprocess
import ipywidgets.widgets as widgets
from ipywidgets import interact

def brightness_change(brightness):
    subprocess.check_output(["v4l2-ctl", "-d", "1", "-c", "brightness={}".format(brightness)])
    
interact(brightness_change, brightness=widgets.IntSlider(min=0, max=255, step=1, value=128));

def contrast_change(contrast):
    subprocess.check_output(["v4l2-ctl", "-d", "1", "-c", "contrast={}".format(contrast)])
interact(contrast_change, contrast=widgets.IntSlider(min=0, max=255, step=1, value=128));

display(ipywidgets.HBox([image, image2]))

## 关闭两个摄像头

In [4]:
camera.unobserve_all()
camera.stop()

usb_camera.unobserve_all()
usb_camera.stop()